# Nonlinear Equations

## Introduction

Nonlinear equations are ubiquitous in scientific computing. In power systems,
the AC power flow equations are nonlinear equations.

The standard form of nonlinear equations is:

$$
f(x) = 0
$$

where `f` is a nonlinear function and `x` is the variable. In other
words, solving nonlinear equations means finding the roots of a function. This
function is called the **residual** function, which will be small enough (zero
at the tolerance level) when the solution is found. 






## Residual Function

The way to represent a nonlinear equation is to define the residual function.
This is straightforward for simple problems in Python.

Consider this problem from `scipy.optimize.fsolve` documentation:

$$
x_0 \cos(x_1) = 4
$$

$$
x_1 x_0 - x_1 = 5
$$

we can define the residual function as:


In [8]:
import numpy as np


def residual(x):
    return [x[0]*np.cos(x[1]) - 4,
            x[0]*x[1] - x[1] - 5]


Note that for the function argument `x`, it is implicitly a 1D NumPy array. 

## Root-Finding with `fsolve`

One of the most common methods to solve nonlinear equations is the
[Newton-Raphson method](https://en.wikipedia.org/wiki/Newton%27s_method). In
practice, for simple problems, we can use off-the-shelf solvers in Python.

`fsolve` is a function in `scipy.optimize` for root-finding. We can check the
docstring by typing `?fsolve` in a Jupyter cell or visiting the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html).

In [11]:
from scipy.optimize import fsolve

x0 = [1, 1]
sol = fsolve(residual, x0)
print(sol)


[6.50409711 0.90841421]


In [14]:
np.isclose(residual(sol), [0.0, 0.0])  # the residual at the solution should be almost 0.0.

array([ True,  True])

## Jacobian Matrix

SciPy's `fsolve` uses the [Powell's hybrid
method](https://en.wikipedia.org/wiki/Powell%27s_dog_leg_method), which combines
the Newton-Raphson method with gradient descent. They require the Jacobian
matrix, which is the first-order derivatives of the residual function w.r.t the
unknown variables.

Calculating the Jacobian matrix is computationally expensive. By default,
`fsolve` uses the `hybrd` method in MINPACK, which calculates the Jacobian
matrix using finite differences. For each column of the Jacobian matrix, two
evaluations of the residual functions are needed, resulting in a total of `2N`
residual evaluations for an N-dimensional problem.

Providing the Jacobian matrix can speed up the calculation. This is one of the
reasons why texts on power system analysis always show Jacobian matrix. 

`fsolve` allows specifying the function that provides the Jacobian matrix
through the `fprime` argument.

To clarify, the Jacobian function is the first-order derivatives of the
**residual function** w.r.t the unknown variables. It is not necessarily the
derivative of the **original function** w.r.t the unknown variables.

In this case, the residual function is:

$$
r_0 = 
\begin{bmatrix}
x_0 \cos(x_1) - 4 \\
x_0 x_1 - x_1 - 5
\end{bmatrix}
$$

And the unkonwns are

$$
x = \begin{bmatrix}
x_0 \\
x_1
\end{bmatrix}
$$

Therefore, the **analytical** Jacobian matrix is:

$$
J = \begin{bmatrix}
\frac{\partial r_0}{\partial x_0} & \frac{\partial r_0}{\partial x_1} \\
\frac{\partial r_1}{\partial x_0} & \frac{\partial r_1}{\partial x_1}
\end{bmatrix} = 
\begin{bmatrix}
\cos(x_1) & -x_0 \sin(x_1) \\
x_1 & x_0 - 1
\end{bmatrix}
$$

It is called the analytical Jacobian matrix because it is derived based on
calculus. 


In [16]:
def jacobian(x):
    return [[np.cos(x[1]), -x[0]*np.sin(x[1])],
            [x[1], x[0] - 1]]


sol = fsolve(residual, x0, fprime=jacobian)
print(sol)

np.isclose(residual(sol), [0.0, 0.0])  # the residual at the solution should be almost 0.0.


[6.50409711 0.90841421]


array([ True,  True])

In this small example, we cannot see the performance gain from providing the
Jacobian matrix. However, this will be illustrated by a power flow problem.

## Power Flow Problem

## Custom Solver